Library

In [ ]:
import os
import numpy as np
import random
import pydicom
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.utils as vutils
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
from PIL import Image, ImageOps
import pandas as pd
from skimage.metrics import mean_squared_error, peak_signal_noise_ratio, structural_similarity as ssim
import lpips
from scipy.stats import wasserstein_distance
from skimage.color import rgb2lab
from skimage.filters import gabor
from scipy.spatial import distance
from sklearn.metrics.pairwise import cosine_similarity

Custom dataset

In [ ]:
transform_pipeline = transforms.Compose([
    transforms.RandomAffine(degrees=20, scale=(0.90, 1.10)),
    transforms.ToTensor(),
])

def visualize_patch(patch, title):
    plt.figure(figsize=(4, 4))
    plt.imshow(patch, cmap='gray')
    plt.title(title)
    plt.colorbar()
    plt.show()
    
    
def normalize_patch(difference_patch):
    min_val = np.min(difference_patch)
    max_val = np.max(difference_patch)
    
    # Avoid division by zero by adding a small constant (e.g., 1e-5) or
    # handle the case where max and min values are equal
    if max_val - min_val > 1e-5:
        normalized_diff_patch = (difference_patch - min_val) / (max_val - min_val)
    else:
        normalized_diff_patch = np.zeros(difference_patch.shape, dtype=np.float32)
    return normalized_diff_patch


class CTDataSet(Dataset):
    def __init__(self, root_dir, patient_ids, patch_size=512, sampling_ratio=1, transform=None):

        self.root_dir = root_dir
        self.patient_ids = patient_ids
        self.patch_size = patch_size
        self.sampling_ratio = sampling_ratio
        self.image_pairs = []
        self.transform = transform
        
        for patient_id in patient_ids:
            pre_path = os.path.join(root_dir, patient_id, "POST VUE")
            post_path = os.path.join(root_dir, patient_id, "POST STD")

            for filename in sorted(os.listdir(pre_path)):
                if filename.endswith('.jpg') and '0001.jpg' <= filename <= '0500.jpg':
                    pre_image_path = os.path.join(pre_path, filename)
                    post_image_path = os.path.join(post_path, filename)
                    self.image_pairs.append((pre_image_path, post_image_path))

    def __len__(self):
        total_patches_per_image_pair = ((512 // self.patch_size) ** 2) * self.sampling_ratio
        return int(len(self.image_pairs) * total_patches_per_image_pair)

    def load_jpg_image_patch(self, image_path, start_x, start_y):
        image = Image.open(image_path)
        patch = image.crop((start_x, start_y, start_x + self.patch_size, start_y + self.patch_size))
        patch = np.array(patch, dtype=np.float32)

        min_val = np.min(patch)
        max_val = np.max(patch)
        if max_val - min_val > 0:
            patch = (patch - min_val) / (max_val - min_val)
        else:
            patch = patch - min_val

        return patch

    def __getitem__(self, idx):
        image_pair_idx = idx % len(self.image_pairs)
        pre_image_path, post_image_path = self.image_pairs[image_pair_idx]

        max_x = 512 - self.patch_size
        max_y = 512 - self.patch_size
        x = random.randint(0, max_x)
        y = random.randint(0, max_y)

        pre_image_patch = self.load_jpg_image_patch(pre_image_path, x, y)
        post_image_patch = self.load_jpg_image_patch(post_image_path, x, y)
        
        # Create the binary contrast mask
        contrast_mask = (post_image_patch >= 0.9).astype(np.float32)

        # Convert patches to tensors and add channel dimension
        pre_image_patch_tensor = torch.from_numpy(pre_image_patch).unsqueeze(0)
        post_image_patch_tensor = torch.from_numpy(post_image_patch).unsqueeze(0)
        contrast_mask = (post_image_patch >= 0.9).astype(np.float32)

        # Apply the same transformation to all patches
        if self.transform:
            seed = np.random.randint(2147483647)  # Get a random seed so that we can reproducibly do the transforms.
            torch.manual_seed(seed)  # Apply this seed to img transforms
            pre_image_patch = self.transform(Image.fromarray(pre_image_patch))
            
            torch.manual_seed(seed)  # Apply this seed to img transforms
            post_image_patch = self.transform(Image.fromarray(post_image_patch))

            torch.manual_seed(seed)  # Apply this seed to mask transforms
            contrast_mask = self.transform(Image.fromarray(contrast_mask))

        return pre_image_patch, post_image_patch, contrast_mask


Model

In [ ]:
class UNet2DGenerator(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet2DGenerator, self).__init__()

        # Encoder
        self.enc1 = self.conv_block(in_channels, 64)
        self.enc2 = self.conv_block(64, 128)
        self.enc3 = self.conv_block(128, 256)
        self.enc4 = self.conv_block(256, 512)
        self.enc5 = self.conv_block(512, 512)
        self.enc6 = self.conv_block(512, 512)

        # Decoder
        self.dec1 = self.conv_block(512 + 512, 512)  # Skip connection 추가
        self.dec2 = self.conv_block(512 + 512, 512)  # Skip connection 추가
        self.dec3 = self.conv_block(512 + 256, 256)  # Skip connection 추가
        self.dec4 = self.conv_block(256 + 128, 128)  # Skip connection 추가
        self.dec5 = self.conv_block(128 + 64, 64)    # Skip connection 추가

        # Final Output
        self.out = nn.Conv2d(64, out_channels, kernel_size=3, padding=1)

    def forward(self, x):
        # Encoder
        e1 = self.enc1(x)
        e2 = self.enc2(F.max_pool2d(e1, 2))
        e3 = self.enc3(F.max_pool2d(e2, 2))
        e4 = self.enc4(F.max_pool2d(e3, 2))
        e5 = self.enc5(F.max_pool2d(e4, 2))
        e6 = self.enc6(F.max_pool2d(e5, 2))

        # Decoder with skip connections
        d1 = self.dec1(torch.cat([F.interpolate(e6, scale_factor=2, mode='nearest'), e5], 1))
        d2 = self.dec2(torch.cat([F.interpolate(d1, scale_factor=2, mode='nearest'), e4], 1))
        d3 = self.dec3(torch.cat([F.interpolate(d2, scale_factor=2, mode='nearest'), e3], 1))
        d4 = self.dec4(torch.cat([F.interpolate(d3, scale_factor=2, mode='nearest'), e2], 1))
        d5 = self.dec5(torch.cat([F.interpolate(d4, scale_factor=2, mode='nearest'), e1], 1))

        # Final output
        out = self.out(d5)
        return torch.sigmoid(out)

    @staticmethod
    def conv_block(in_channels, out_channels):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )


class Discriminator2D(nn.Module):
    def __init__(self, in_channels=2):
        super(Discriminator2D, self).__init__()
        self.model = nn.Sequential(
            self.discriminator_block(in_channels, 64, normalization=False),
            self.discriminator_block(64, 128),
            self.discriminator_block(128, 256),
            self.discriminator_block(256, 512),
            # 마지막 컨볼루션 레이어의 커널 크기를 1x1로 조정
            nn.Conv2d(512, 1, kernel_size=1, stride=1, padding=0),
            nn.AdaptiveAvgPool2d(1),  # 모든 입력을 (1, 1) 크기로 평균 풀링

            nn.Sigmoid()
        )
    
    def discriminator_block(self, in_channels, out_channels, normalization=True):
        layers = [nn.Conv2d(in_channels, out_channels, kernel_size=4, stride=2, padding=1)]
        if normalization:
            layers.append(nn.BatchNorm2d(out_channels))
        layers.append(nn.LeakyReLU(0.2, inplace=True))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        for i, layer in enumerate(self.model):
            x = layer(x)
        return x.view(-1, 1)


def weights_init(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        nn.init.normal_(m.weight, 0.0, 0.02)
    elif isinstance(m, nn.BatchNorm2d):
        nn.init.normal_(m.weight, 1.0, 0.02)
        nn.init.constant_(m.bias, 0)

Training & testing

In [ ]:
batch_size = 64

# Custom loss function
def custom_loss(generated, target, mask, lambda_contrast=10):
    l1_loss = nn.L1Loss(reduction='none')
    # Apply mask with higher weight to contrast areas
    loss = l1_loss(generated, target) * (1 + mask * lambda_contrast)
    return loss.mean()


def load_model_weights(model, weight_path, primary_device):
    if os.path.exists(weight_path):
        # Load the saved weights
        state_dict = torch.load(weight_path, map_location=primary_device)
        
        # Adjust for DataParallel prefix
        if isinstance(model, torch.nn.DataParallel):
            # No adjustment needed for DataParallel model
            model.load_state_dict(state_dict)
        else:
            # Remove 'module.' prefix for single GPU model
            new_state_dict = {k.replace('module.', ''): v for k, v in state_dict.items()}
            model.load_state_dict(new_state_dict)
        
        print(f"Loaded weights from {weight_path}")
    else:
        print(f"No weights found at {weight_path}, starting from scratch.")



def visualize_images(dataset, G, num_images=10):
    plt.figure(figsize=(20, num_images * 5))  

    for i, idx in enumerate(random.sample(range(len(dataset)), num_images), start=1):
        pre_img, post_img, contrast_mask = dataset[idx]

        
        if isinstance(pre_img, np.ndarray):
            pre_img = torch.from_numpy(pre_img).unsqueeze(0) 
        if isinstance(post_img, np.ndarray):
            post_img = torch.from_numpy(post_img).unsqueeze(0)
        if isinstance(contrast_mask, np.ndarray):
            contrast_mask = torch.from_numpy(contrast_mask).unsqueeze(0)

        pre_img = pre_img.float().unsqueeze(0)  
        pre_img = pre_img.to('cuda')  

        with torch.no_grad():  
            generated_img = G(pre_img).squeeze(0).cpu() 

        # Displaying the PRE CT Patch
        plt.subplot(num_images, 4, i * 4 - 3)
        plt.imshow(pre_img.squeeze().cpu().numpy(), cmap='gray')  
        plt.title(f'PRE CT Patch {i}')
        plt.axis('off')

        # Displaying the GENERATED POST CT Patch
        plt.subplot(num_images, 4, i * 4 - 2)
        plt.imshow(generated_img.squeeze().numpy(), cmap='gray')
        plt.title(f'Generated POST CT Patch {i}')
        plt.axis('off')

        # Displaying the REAL POST CT Patch
        plt.subplot(num_images, 4, i * 4 - 1)
        plt.imshow(post_img.squeeze().cpu().numpy(), cmap='gray')
        plt.title(f'Real POST CT Patch {i}')
        plt.axis('off')

        # Displaying the CONTRAST MASK
        plt.subplot(num_images, 4, i * 4)
        plt.imshow(contrast_mask.squeeze().cpu().numpy(), cmap='gray')
        plt.title(f'Contrast Mask {i}')
        plt.axis('off')

    plt.tight_layout()
    plt.show()



# Dataset splitting
root_path = "/workspace/kyt3426/project_chest_CT_GAN/pilot_data_jpg_dualCT"
all_patient_ids = [name for name in os.listdir(root_path) if os.path.isdir(os.path.join(root_path, name))]


# 훈련 세트 할당
train_patient_ids = [
    'KP-0003', 'KP-0004', 'KP-0007', 'KP-0010', 'KP-0012', 'KP-0013', 'KP-0014', 'KP-0015', 'KP-0016', 
    'KP-0018', 'KP-0019', 'KP-0020', 'KP-0021', 'KP-0023', 'KP-0024', 'KP-0026', 'KP-0027', 'KP-0030', 
    'KP-0031', 'KP-0032', 'KP-0033', 'KP-0034', 'KP-0035', 'KP-0036', 'KP-0037', 'KP-0038', 'KP-0039', 
    'KP-0040', 'KP-0041', 'KP-0042', 'KP-0043', 'KP-0046', 'KP-0047', 'KP-0048', 'KP-0049', 'KP-0050', 
    'KP-0051', 'KP-0052', 'KP-0053', 'KP-0055'
]

# 테스트 세트 할당
test_patient_ids = ['KP-0056']

train_dataset = CTDataSet(root_path, train_patient_ids, transform=transform_pipeline)
test_dataset = CTDataSet(root_path, test_patient_ids, transform=None)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

primary_device = "cuda:0"


G = UNet2DGenerator(in_channels=1, out_channels=1).to(primary_device)
D = Discriminator2D(in_channels=2).to(primary_device)  # Input channels should be 2 since we're concatenating source and target.
 

G.apply(weights_init)
D.apply(weights_init)


criterion = nn.BCEWithLogitsLoss()
l1_criterion = nn.L1Loss()
optimizer_G = torch.optim.Adam(G.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(D.parameters(), lr=0.0002, betas=(0.5, 0.999))


device_list = ['cuda:0', 'cuda:1', 'cuda:2', 'cuda:3', 'cuda:4', 'cuda:5', 'cuda:6', 'cuda:7']
if torch.cuda.device_count() > 1:
    G = nn.DataParallel(G, device_ids=device_list)
    D = nn.DataParallel(D, device_ids=device_list)


G.to(primary_device)
D.to(primary_device)

# # Load pre-trained weights if available
load_model_weights(G, 'patch_complicated_lc1_onlynormal_512x512_customloss_N20_Dual_CT_attention_G_epoch_200.pth', primary_device)
load_model_weights(D, 'patch_complicated_lc1_onlynormal_512x512_customloss_N20_Dual_CT_attention_D_epoch_200.pth', primary_device)


num_epochs = 200000
lambda_l1 = 100


start_epoch = 0  # Start from the next epoch after the loaded one

# 학습 루프
for epoch in range(start_epoch, num_epochs):
    G.train()
    
    progress_bar = tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc=f'Epoch {epoch+1}/{num_epochs}')

    
    for i, (real_pre_images, real_post_images, contrast_masks) in progress_bar:
    
        real_pre_images, real_post_images = real_pre_images.to(primary_device), real_post_images.to(primary_device)
        contrast_masks = contrast_masks.to(primary_device)  # Make sure masks are on the correct device

        current_batch_size = real_pre_images.size(0)

        # Discriminator
        D.zero_grad()

        # Real images
        real_labels = torch.ones(current_batch_size, 1).to(primary_device)
        real_combined = torch.cat([real_pre_images, real_post_images], dim=1) 
        outputs = D(real_combined)
        loss_real = criterion(outputs, real_labels)

        # Fake images
        fake_post_images = G(real_pre_images)
        fake_combined = torch.cat([real_pre_images, fake_post_images.detach()], dim=1)
        outputs = D(fake_combined)
        fake_labels = torch.zeros(current_batch_size, 1).to(primary_device)
        loss_fake = criterion(outputs, fake_labels)

        loss_d = loss_real + loss_fake
        loss_d.backward()
        optimizer_D.step()

        # Generator 
        # Generate fake images
        fake_post_images = G(real_pre_images)

        # Combine with real pre-images for the discriminator
        fake_combined = torch.cat([real_pre_images, fake_post_images], dim=1)
        outputs = D(fake_combined)

        # Calculate GAN loss for the generator
        loss_g_gan = criterion(outputs, torch.ones(current_batch_size, 1).to(primary_device))

        # Calculate custom L1 loss with contrast mask
        loss_g_custom = custom_loss(fake_post_images, real_post_images, contrast_masks, lambda_contrast=1)
        
        # Combined generator loss
        loss_g = loss_g_gan + lambda_l1 * loss_g_custom

        loss_g.backward()
        optimizer_G.step()
        
        progress_bar.set_postfix(Loss_D=loss_d.item(), Loss_G=loss_g.item())

    # print(f'Epoch [{epoch+1}/{num_epochs}], Loss D: {loss_d.item():.4f}, Loss G: {loss_g.item():.4f}')


    if (epoch + 1) % 50 == 0:
        G.eval()
        with torch.no_grad():
            visualize_images(test_dataset, G, num_images=10)
            

    if (epoch + 1) % 50 == 0:
        torch.save(G.state_dict(), f'patch_complicated_lc1_onlynormal_512x512_customloss_N20_Dual_CT_attention_G_epoch_{epoch+1}.pth')
        torch.save(D.state_dict(), f'patch_complicated_lc1_onlynormal_512x512_customloss_N20_Dual_CT_attention_D_epoch_{epoch+1}.pth')
        print(f'Model weights saved for epoch {epoch+1}')
        

Inference

In [ ]:
# Model definition
G = UNet2DGenerator(in_channels=1, out_channels=1)

# Load the perceptual similarity metric model for LPIPS
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
lpips_model = lpips.LPIPS(net='alex').to(device)

# Weight file path
weight_path = '/workspace/kyt3426/project_chest_CT_GAN/patch_complicated_lc0.5_onlynormal_512x512_customloss_N20_Dual_CT_attention_G_epoch_300.pth'

# Function to load model weights
def load_model_weights(model, weight_path):
    state_dict = torch.load(weight_path, map_location='cpu')  # Load to CPU
    new_state_dict = {k.replace('module.', ''): v for k, v in state_dict.items()}
    model.load_state_dict(new_state_dict)

# Load weights into the model
load_model_weights(G, weight_path)

# Ensure the model is on the GPU if available
G.to(device)

# EMD Calculation
def calculate_emd(img1, img2):
    img1_flat = img1.flatten()
    img2_flat = img2.flatten()
    emd_value = wasserstein_distance(img1_flat, img2_flat)
    return emd_value

# Color-Based Similarity Calculation
def calculate_color_similarity(img1, img2):
    if len(img1.shape) == 2 and len(img2.shape) == 2:
        return 0.0
    
    img1_lab = rgb2lab(img1)
    img2_lab = rgb2lab(img2)
    
    color_dist = distance.euclidean(img1_lab.flatten(), img2_lab.flatten())
    return color_dist

# Texture-Based Similarity Calculation using Gabor Filters
def calculate_texture_similarity(img1, img2):
    filt_real1, _ = gabor(img1, frequency=0.6)
    filt_real2, _ = gabor(img2, frequency=0.6)
    texture_dist = np.linalg.norm(filt_real1 - filt_real2)
    return texture_dist

# Cosine Similarity Calculation
def calculate_cosine_similarity(img1, img2):
    img1_flat = img1.flatten().reshape(1, -1)
    img2_flat = img2.flatten().reshape(1, -1)
    cosine_sim = cosine_similarity(img1_flat, img2_flat)[0][0]
    return cosine_sim

# Euclidean Distance Calculation
def calculate_euclidean_distance(img1, img2):
    img1_flat = img1.flatten()
    img2_flat = img2.flatten()
    # Euclidean Distance 계산
    euclidean_dist = np.linalg.norm(img1_flat - img2_flat)
    return euclidean_dist

# Function to calculate all metrics, including cosine similarity and euclidean distance
def calculate_metrics(img1, img2):
    mae = np.mean(np.abs(img1 - img2))  # MAE
    psnr = peak_signal_noise_ratio(img1, img2)  # PSNR
    ms_ssim = ssim(img1, img2, data_range=1.0, multichannel=False)  # MS-SSIM
    
    # LPIPS
    img1_tensor = torch.from_numpy(img1).float().unsqueeze(0).unsqueeze(0).to(device)
    img2_tensor = torch.from_numpy(img2).float().unsqueeze(0).unsqueeze(0).to(device)
    lpips_value = lpips_model(img1_tensor, img2_tensor).item()
    
    emd_value = calculate_emd(img1, img2)  # EMD
    color_similarity = calculate_color_similarity(img1, img2)  # Color Similarity
    texture_similarity = calculate_texture_similarity(img1, img2)  # Texture Similarity
    cosine_sim = calculate_cosine_similarity(img1, img2)  # Cosine Similarity
    euclidean_dist = calculate_euclidean_distance(img1, img2)  # Euclidean Distance

    return mae, psnr, ms_ssim, lpips_value, emd_value, color_similarity, texture_similarity, cosine_sim, euclidean_dist

def inference_and_evaluate(model, input_folder, output_folder, patch_size=512, stride=1):
    model.eval()  
    results = []
    
    for patient_id in os.listdir(input_folder):
        patient_path = os.path.join(input_folder, patient_id)
        if not os.path.isdir(patient_path):
            continue

        post_vue_path = os.path.join(patient_path, 'POST VUE')  
        post_std_path = os.path.join(patient_path, 'POST STD')  
        
        if not os.path.exists(post_vue_path) or not os.path.exists(post_std_path):
            continue

        for image_file in os.listdir(post_vue_path):
            if not image_file.endswith('.jpg'):
                continue

            # Load non-contrast (POST VUE) image
            non_contrast_img_path = os.path.join(post_vue_path, image_file)
            non_contrast_img = np.array(Image.open(non_contrast_img_path).convert('L'), dtype=np.float32)
            non_contrast_img = (non_contrast_img - np.min(non_contrast_img)) / (np.max(non_contrast_img) - np.min(non_contrast_img))  # Normalize

            # Load ground truth contrast (POST STD) image
            contrast_img_path = os.path.join(post_std_path, image_file)
            contrast_img = np.array(Image.open(contrast_img_path).convert('L'), dtype=np.float32)
            contrast_img = (contrast_img - np.min(contrast_img)) / (np.max(contrast_img) - np.min(contrast_img))  # Normalize

            # Inference: Generate contrast CT from non-contrast CT (using GAN)
            image_tensor = torch.from_numpy(non_contrast_img).unsqueeze(0).unsqueeze(0).to(device)
            with torch.no_grad():
                generated_img_tensor = model(image_tensor)
            generated_img = generated_img_tensor.squeeze().cpu().numpy()
            generated_img = (generated_img - np.min(generated_img)) / (np.max(generated_img) - np.min(generated_img))  # Normalize

            # Calculate metrics for (1) GAN-generated vs Ground Truth
            mae_gen, psnr_gen, ms_ssim_gen, lpips_gen, emd_gen, color_sim_gen, texture_sim_gen, cosine_sim_gen, euclidean_dist_gen = calculate_metrics(generated_img, contrast_img)
            
            # Calculate metrics for (2) Non-contrast vs Ground Truth
            mae_nc, psnr_nc, ms_ssim_nc, lpips_nc, emd_nc, color_sim_nc, texture_sim_nc, cosine_sim_nc, euclidean_dist_nc = calculate_metrics(non_contrast_img, contrast_img)

            # Append results for CSV
            results.append({
                'Patient ID': patient_id,
                'Slice': image_file,
                'MAE_GAN_vs_GT': mae_gen,
                'PSNR_GAN_vs_GT': psnr_gen,
                'MS-SSIM_GAN_vs_GT': ms_ssim_gen,
                'LPIPS_GAN_vs_GT': lpips_gen,
                'EMD_GAN_vs_GT': emd_gen,
                'Color_Similarity_GAN_vs_GT': color_sim_gen,
                'Texture_Similarity_GAN_vs_GT': texture_sim_gen,
                'Cosine_Similarity_GAN_vs_GT': cosine_sim_gen,
                'Euclidean_Distance_GAN_vs_GT': euclidean_dist_gen,
                'MAE_NC_vs_GT': mae_nc,
                'PSNR_NC_vs_GT': psnr_nc,
                'MS-SSIM_NC_vs_GT': ms_ssim_nc,
                'LPIPS_NC_vs_GT': lpips_nc,
                'EMD_NC_vs_GT': emd_nc,
                'Color_Similarity_NC_vs_GT': color_sim_nc,
                'Texture_Similarity_NC_vs_GT': texture_sim_nc,
                'Cosine_Similarity_NC_vs_GT': cosine_sim_nc,
                'Euclidean_Distance_NC_vs_GT': euclidean_dist_nc
            })
    
    # Save results to CSV
    df = pd.DataFrame(results)
    output_csv_path = os.path.join(output_folder, 'evaluation_results_updated.csv')
    df.to_csv(output_csv_path, index=False)
    print(f"Results saved to {output_csv_path}")